In [0]:
# Catalog and schema
spark.sql("USE CATALOG finance_catalog")
spark.sql("USE finance_catalog.silver")

from pyspark.sql.functions import col, trim, upper

#  Bronze data 
df_bronze = spark.read.table("finance_catalog.bronze.transactions")

# Minimum data cleaning rules
df_silver = (
    df_bronze
    .filter(col("amount").isNotNull())      
    .filter(col("amount") > 0)              
    .filter(col("customer_id").isNotNull()) 
    .withColumn("origin_country", upper(trim(col("origin_country"))))
    .withColumn("destination_country", upper(trim(col("destination_country"))))
    .withColumn("currency", trim(col("currency")))
    .withColumn("merchant", trim(col("merchant")))
)

# Write to Silver (Delta Table)
(
    df_silver.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("finance_catalog.silver.transactions")
)

print("Silver table successfully created :D")


---------------------------------------------------------------------------
NumberFormatException                     Traceback (most recent call last)
File <command-4981515644780274>, line 28
     11 df_silver = (
     12     df_bronze
     13     .filter(col("amount").isNotNull())      
   (...)
     19     .withColumn("merchant", trim(col("merchant")))
     20 )
     22 # Write to Silver (Delta Table)
     23 (
     24     df_silver.write
     25     .format("delta")
     26     .mode("overwrite")
     27     .option("overwriteSchema", "true")
---> 28     .saveAsTable("finance_catalog.silver.transactions")
     29 )
     31 print("Silver table successfully created :D")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:713, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    711 self._write.table_name = name
    712 self._write.table_save_method = "save_as_table"
--> 713 _, _, ei = self._spark.client.execute_comma

In [0]:
%sql
SHOW TABLES IN finance_catalog.silver;

SELECT COUNT(*) FROM finance_catalog.silver.transactions;
